In [1]:
import networkx as nx
import seaborn as sns
import pandas as pd
import glob
import ast

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
filepath = r"C:\Users\Proprietario\Desktop\SLAPP\SLAPP3-master\6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX\Analisi 2" # use your path
all_files = glob.glob(filepath + "/*.txt")
G = {}
j = 0

for filename in all_files:
    G[j]=nx.DiGraph()
    fh=open(filename,'r')
    for i, line in enumerate(fh):
        s=line.strip()
        s=s.strip(";").split()
        if (i>6) and (s[0] != 'label') and (s[0] != '}') and (s[1] != '[style=filled,'):
            origin=int(s[0])
            dest=int(s[2])
            G[j].add_edge(origin,dest)
    fh.close()
    j += 1

In [3]:
for i in range (len(G)):
    print('The network has', len(G[i]), 'nodes and', len(G[i].edges()), 'links.')

The network has 32557 nodes and 101739 links.
The network has 51624 nodes and 307685 links.
The network has 14844 nodes and 39915 links.
The network has 32149 nodes and 107183 links.
The network has 52047 nodes and 270415 links.
The network has 38495 nodes and 163601 links.
The network has 31240 nodes and 76780 links.
The network has 18381 nodes and 58037 links.
The network has 48751 nodes and 182494 links.
The network has 54561 nodes and 278004 links.


In [4]:
for i in range (len(G)):
    print("Is the graph simply connected?", nx.is_strongly_connected(G[i]))

Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False
Is the graph simply connected? False


### Show the components of the graph

In [5]:
for i in range (len(G)):
    print("The graph has", nx.number_strongly_connected_components(G[i]),"connected components")

The graph has 32557 connected components
The graph has 51624 connected components
The graph has 14844 connected components
The graph has 32149 connected components
The graph has 52047 connected components
The graph has 38495 connected components
The graph has 31240 connected components
The graph has 18381 connected components
The graph has 48751 connected components
The graph has 54561 connected components


### Extract the largest Connected Component as a subgraph

In [6]:
graphs = {}

for i in range (len(G)):
    graphs[i] = []
    for c in nx.strongly_connected_components(G[i]):
        graphs[i].append(G[i].subgraph(c))

In [7]:
H = {}

for i in range (len(G)):
    H[i] = graphs[i][0]

In [8]:
for i in range (len(G)):
    print(len(H[i]))

1
1
1
1
1
1
1
1
1
1


In [9]:
for i in range (len(G)):
    print(len(G[i])-len(H[i]))

32556
51623
14843
32148
52046
38494
31239
18380
48750
54560


In [10]:
for i in range (len(G)):
    print("Check that the graph is now connected")
    print(nx.number_strongly_connected_components(H[i]))

Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1
Check that the graph is now connected
1


## Clustering

In [11]:
files = glob.glob(filepath + "/*.csv")
dataset = {}
j = 0

for filename in files:
    dataset[j] = pd.read_csv(filename, encoding="utf8")
    j += 1

In [12]:
for i in range(len(dataset)):
    print(dataset[i].head)

<bound method NDFrame.head of         id_autore agent_type     score
0           30177     voters  0.529756
1            8438     voters  0.209021
2           43788     voters  0.510521
3           22840     voters  0.334266
4           75887     voters  0.746125
...           ...        ...       ...
99996         686     voters  0.035972
99997       26625     voters  0.532284
99998       53365     voters  0.684630
99999       16833     voters  0.285604
100000      35710     voters  0.540229

[100001 rows x 3 columns]>
<bound method NDFrame.head of         id_autore agent_type     score
0           83344     voters  0.767930
1            9325     voters  0.294695
2           30626     voters  0.487381
3           34093     voters  0.524170
4           51799     voters  0.577045
...           ...        ...       ...
99996       16496     voters  0.296982
99997       38584     voters  0.460588
99998       18049     voters  0.370242
99999       92859     voters  0.850553
100000      592

In [49]:
N_nodes = {}
N_edges = {}
fake_news = {}
bias_right = {}
right = {}
right_leaning = {}
center_news = {}
left_leaning = {}
left = {}
bias_left = {}

for j in range (len(G)):
    N_nodes[str(j)] = [0]*8
    N_edges[str(j)] = [0]*8
    fake_news[j] = nx.DiGraph()
    bias_right[j] = nx.DiGraph()
    right[j] = nx.DiGraph()
    right_leaning[j] = nx.DiGraph()
    center_news[j] = nx.DiGraph()
    left_leaning[j] = nx.DiGraph()
    left[j] = nx.DiGraph()
    bias_left[j] = nx.DiGraph()
    
    for i in range (len(dataset[j])):
        if G[j].has_node(dataset[j]['id_autore'][i]):
            if dataset[j]['score'][i] < 0.1:
                N_nodes[str(j)][0] += 1
                N_edges[str(j)][0] += G[j].degree(dataset[j]['id_autore'][i])
                fake_news[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.2) and (dataset[j]['score'][i] > 0.1):
                N_nodes[str(j)][1] += 1
                N_edges[str(j)][1] += G[j].degree(dataset[j]['id_autore'][i])
                bias_right[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.3) and (dataset[j]['score'][i] > 0.2):
                N_nodes[str(j)][2] += 1
                N_edges[str(j)][2] += G[j].degree(dataset[j]['id_autore'][i])
                right[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.45) and (dataset[j]['score'][i] > 0.3):
                N_nodes[str(j)][3] += 1
                N_edges[str(j)][3] += G[j].degree(dataset[j]['id_autore'][i])
                right_leaning[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.55) and (dataset[j]['score'][i] > 0.45):
                N_nodes[str(j)][4] += 1
                N_edges[str(j)][4] += G[j].degree(dataset[j]['id_autore'][i])
                center_news[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.7) and (dataset[j]['score'][i] > 0.55):
                N_nodes[str(j)][5] += 1
                N_edges[str(j)][5] += G[j].degree(dataset[j]['id_autore'][i])
                left_leaning[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.8) and (dataset[j]['score'][i] > 0.7):
                N_nodes[str(j)][6] += 1
                N_edges[str(j)][6] += G[j].degree(dataset[j]['id_autore'][i])
                left[j].add_node(dataset[j]['id_autore'][i])
            if (dataset[j]['score'][i] < 0.9) and (dataset[j]['score'][i] > 0.8):
                N_nodes[str(j)][7] += 1
                N_edges[str(j)][7] += G[j].degree(dataset[j]['id_autore'][i])
                bias_left[j].add_node(dataset[j]['id_autore'][i])

In [14]:
ok = True

In [15]:
if ok == True:
    for j in range (1):
        for i in fake_news[j+1].node():
            for k in fake_news[j+1].node():
                if G[j+1].has_edge(i,k)==True:
                    fake_news[j+1].add_edge(i,k)

In [16]:
if ok == True:
    for j in range (1):
        for i in bias_right[j+1].node():
            for k in bias_right[j+1].node():
                if G[j+1].has_edge(i,k)==True:
                    bias_right[j+1].add_edge(i,k)

In [17]:
if ok == True:
    for j in range (1):
        for i in right[j+1].node():
            for k in right[j+1].node():
                if G[j+1].has_edge(i,k)==True:
                    right[j+1].add_edge(i,k)

In [18]:
if ok == True:
    for j in range (1):
        for i in right_leaning[j+1].node():
            for k in right_leaning[j+1].node():
                if G[j+1].has_edge(i,k)==True:
                    right_leaning[j+1].add_edge(i,k)

In [19]:
if ok == True:
    for j in range (1):
        for i in center_news[j+1].node():
            for k in center_news[j+1].node():
                if G[j+1].has_edge(i,k)==True:
                    center_news[j+1].add_edge(i,k)

In [20]:
if ok == True:
    for j in range (1):
        for i in left_leaning[j+1].node():
            for k in left_leaning[j+1].node():
                if G[j+1].has_edge(i,k)==True:
                    left_leaning[j+1].add_edge(i,k)

In [21]:
if ok == True:
    for j in range (1):
        for i in left[j+1].node():
            for k in left[j+1].node():
                if G[j+1].has_edge(i,k)==True:
                    left[j+1].add_edge(i,k)

In [22]:
if ok == True:
    for j in range (1):
        for i in bias_left[j+1].node():
            for k in bias_left[j+1].node():
                if G[j+1].has_edge(i,k)==True:
                    bias_left[j+1].add_edge(i,k)

In [23]:
if ok == True:
    for j in range (1):
        print('Fake-News Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(fake_news[j+1]),
              "connected components")

    for j in range (1):
        print('Bias-Right Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(bias_right[j+1]),
              "connected components")

    for j in range (1):
        print('Right Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(right[j+1]),
              "connected components")
    
    for j in range (1):
        print('Right-Leaning Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(right_leaning[j+1]),
              "connected components")
    
    for j in range (1):
        print('Center-News Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(center_news[j+1]),
              "connected components")
        
    for j in range (1):
        print('Left-Leaning Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(left_leaning[j+1]),
              "connected components")
        
    for j in range (1):
        print('Left Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(left[j+1]),
              "connected components")
        
    for j in range (1):
        print('Bias-Left Networks:')
        print("The graph has", 
              nx.number_strongly_connected_components(bias_left[j+1]),
              "connected components")

Fake-News Networks:
The graph has 1124 connected components
Bias-Right Networks:
The graph has 6286 connected components
Right Networks:
The graph has 8064 connected components
Right-Leaning Networks:
The graph has 4576 connected components
Center-News Networks:
The graph has 14216 connected components
Left-Leaning Networks:
The graph has 13005 connected components
Left Networks:
The graph has 3747 connected components
Bias-Left Networks:
The graph has 607 connected components


In [50]:
K_mean = {}
voters_type = 8

for j in range (len(G)):
    K_mean[str(j)] = []
    print('\n')
    for i in range (voters_type):
        if N_nodes[str(j)][i] != 0:
            K_mean[str(j)].append(N_edges[str(j)][i]/N_nodes[str(j)][i])
            print("The ", i+1, " K_mean value is: ",K_mean[str(j)][i])
        else:
            K_mean[str(j)].append(str('Node do not exist'))
            print("The ", i+1, " K_mean value is: ",K_mean[str(j)][i])



The  1  K_mean value is:  6.073221757322176
The  2  K_mean value is:  5.4194687289845325
The  3  K_mean value is:  5.477353463587922
The  4  K_mean value is:  4.281036834924966
The  5  K_mean value is:  3.797378060535955
The  6  K_mean value is:  8.580557301487534
The  7  K_mean value is:  385.6904761904762
The  8  K_mean value is:  2.5


The  1  K_mean value is:  55.63256227758007
The  2  K_mean value is:  10.80989500477251
The  3  K_mean value is:  9.297867063492063
The  4  K_mean value is:  9.731206293706293
The  5  K_mean value is:  7.658202025886325
The  6  K_mean value is:  8.580161476355247
The  7  K_mean value is:  17.960768614891915
The  8  K_mean value is:  127.90939044481054


The  1  K_mean value is:  6.068965517241379
The  2  K_mean value is:  3.9151526865094706
The  3  K_mean value is:  3.505880007972892
The  4  K_mean value is:  6.5846736045411545
The  5  K_mean value is:  14.268782383419689
The  6  K_mean value is:  8.0
The  7  K_mean value is:  2.0
The  8  K_mean val

In [25]:
import pydot
from networkx.drawing.nx_pydot import write_dot
oks = False

if oks == True:
    for j in range (1):
        write_dot(fake_news[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/fake-news' + str(j) + '.dot')

    for j in range (1):
        write_dot(bias_right[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/bias-right' + str(j) + '.dot')

    for j in range (1):
        write_dot(right[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/right' + str(j) + '.dot')

    for j in range (1):
        write_dot(right_leaning[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/right-leaning' + str(j) + '.dot')

    for j in range (1):
        write_dot(center_news[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/center-news' + str(j) + '.dot')

    for j in range (1):
        write_dot(left_leaning[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/left-leaning' + str(j) + '.dot')

    for j in range (1):
        write_dot(left[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/left' + str(j) + '.dot')

    for j in range (1):
        write_dot(bias_left[j],'C:/Users/Proprietario/Desktop/SLAPP/SLAPP3-master/6 objectSwarmObserverAgents_AESOP_turtleLib_NetworkX/Analisi 2/bias-left' + str(j) + '.dot')

In [26]:
nx.write_graphml_lxml(fake_news[1], "fake_news.graphml")
nx.write_graphml_lxml(bias_right[1], "bias_right.graphml")
nx.write_graphml_lxml(right[1], "right.graphml")
nx.write_graphml_lxml(right_leaning[1], "right_leaning.graphml")
nx.write_graphml_lxml(center_news[1], "center_news.graphml")
nx.write_graphml_lxml(left_leaning[1], "left_leaning.graphml")
nx.write_graphml_lxml(left[1], "left.graphml")
nx.write_graphml_lxml(bias_left[1], "bias_left.graphml")

In [51]:
for i in range (8):
    k_final = 0
    K_final = []
    
    for j in range (len(G)):
        K_final.append(K_mean[str(j)][i])
    print("The ", i+1, "k-final is: ", np.mean(K_final), "+-", np.std(K_final))

The  1 k-final is:  30.514318898456214 +- 23.23564598548564
The  2 k-final is:  6.766107259348123 +- 2.510508243121859
The  3 k-final is:  5.758734550471618 +- 1.9502735332976482
The  4 k-final is:  6.008792182031149 +- 2.014325787111302
The  5 k-final is:  6.611105358417872 +- 2.9275394448216905
The  6 k-final is:  7.8766814201588975 +- 2.0557189722858134
The  7 k-final is:  84.36354318775693 +- 151.47567138175097
The  8 k-final is:  40.30848759920477 +- 53.78274869220111
